In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

In [ ]:
from os.path import expanduser
import sys
motornet_home = expanduser("~") + "/MotorNet"
sys.path.append(motornet_home)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import motornet as mn
from motornet.plants.muscles import RigidTendonHillMuscleThelen
import plg_tasks

# Disable all GPUS
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

%load_ext autoreload
%autoreload 2

In [ ]:
# SET UP PLANT
pos_lower_bound = np.deg2rad([5, 5])
pos_upper_bound = np.deg2rad([125, 125])
plant = mn.plants.RigidTendonArm26(muscle_type=RigidTendonHillMuscleThelen(), visual_delay=0.07, proprioceptive_delay=0.02, pos_lower_bound=pos_lower_bound, pos_upper_bound=pos_upper_bound)

# SET UP NETWORK
n_units = 49
network = mn.nets.layers.GRUNetwork(plant=plant, n_units=n_units, kernel_regularizer=1e-7, name='network', recurrent_regularizer=1e-5)

# SET UP TASK
task = plg_tasks.Gribble1999(network=network)
#task.loss_weights['cartesian position'] = 1.0
#task.loss_weights['muscle state'] = 5.0

rnn = tf.keras.layers.RNN(cell=network, return_sequences=True, name='RNN')

input_dict = task.get_input_dict_layers()
state0 = task.get_initial_state_layers()
states_out = rnn(input_dict, initial_state=state0)

nn = mn.nets.MotorNetModel(inputs=[input_dict, state0], outputs=states_out, name='model', task=task)
nn.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)

In [ ]:
task.get_losses()


In [ ]:
# TRAIN THE NETWORK to make point-to-point movements to random targets in the arm's workspace
n_t = 100
n_batches = 1024
batch_size = 32
# this callback logs training information for each batch passed, rather than for each epoch.
callbacks = [mn.nets.callbacks.BatchLogger(), mn.nets.callbacks.TrainingPlotter(task, plot_freq=512, plot_trials=1, plot_n_t=n_t)]
for i in range(5):
    [inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=n_batches * batch_size)
    h = nn.fit(x=[inputs, init_states], y=targets, verbose=1, epochs=1, batch_size=batch_size, shuffle=False, callbacks=callbacks)

In [ ]:
def hand_to_joints(xy, L1, L2):
      j = np.zeros(np.shape(xy))
      j[:,:,1] = np.arccos((xy[:,:,0]**2 + xy[:,:,1]**2 - L1**2 - L2**2) / (2*L1*L2))
      j[:,:,0] = np.arctan2(xy[:,:,1],xy[:,:,0]) - np.arctan2(L2*np.sin(j[:,:,1]),L1+(L2*np.cos(j[:,:,1])))
      return j

def plot_simulations(xy, target_xy):
  target_x = target_xy[:, -1, 0]
  target_y = target_xy[:, -1, 1]
  plt.figure()
  nmov,ntime,_ = np.shape(xy)
  for i in range(nmov):
        plt.plot(xy[i,:,0],xy[i,:,1],color="b",linewidth=1.0)
  plt.scatter(target_x, target_y)
  plt.axis("equal")
  plt.xlabel("X (m)")
  plt.ylabel("Y (m)")
  plt.show()

def plot1trial(inputs, results, targets_j, trial=0):
    j_results = results['joint position']
    m_results = results['muscle state']

    plt.figure(figsize=(14, 2.5)).set_tight_layout(True)

    plt.subplot(141)
    plt.plot(j_results[trial, :, 0]*180/np.pi, label='sho')
    plt.plot(j_results[trial, :, 1]*180/np.pi, label='elb')
    plt.plot(targets_j[trial, :, 0]*180/np.pi, '--')
    plt.plot(targets_j[trial, :, 1]*180/np.pi, '--')
    plt.axvline(np.where(inputs["inputs"][trial, :, -1] != 1)[0][0] - network.visual_delay, c='grey')
    plt.legend()
    plt.xlabel('time (ms)')
    plt.ylabel('angle (deg)')

    plt.subplot(142)
    plt.plot(j_results[trial, :, 2], label='sho')
    plt.plot(j_results[trial, :, 3], label='elb')
    plt.legend()
    plt.xlabel('time (ms)')
    plt.ylabel('angle velocity (rad/sec)')

    plt.subplot(143)
    plt.plot(m_results[trial, :, 0, :])
    plt.xlabel('time (ms)')
    plt.ylabel('activation (a.u.)')
    plt.legend(["PEC","DEL","BRA","TRI","BIC","TR2"], loc=1)

    plt.subplot(144)
    plt.plot(m_results[trial, :, -1, :])
    plt.xlabel('time (ms)')
    plt.ylabel('force (N)')
    plt.legend(["PEC","DEL","BRA","TRI","BIC","TR2"], loc=1)

    plt.show()



In [ ]:
# TEST THE NETWORK on the Gribble 1999 movement targets: EXPERIMENT 1 (elbow flexion only)
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=1, experiment=1)
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
# TEST THE NETWORK on the Gribble 1999 movement targets: EXPERIMENT 2 (shoulder flexion only)
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=1, experiment=2)
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
# TEST THE NETWORK on the Gribble 1999 movement targets: EXPERIMENT 3 (combined shoulder and elbow flexion/extension)
[inputs, targets, init_states] = nn.task.generate(n_timesteps=n_t, batch_size=1, experiment=3)
results = nn([inputs, init_states], training=False)

plot_simulations(xy=results["cartesian position"], target_xy=targets)

L1,L2 = nn.task.network.plant.skeleton.L1, task.network.plant.skeleton.L2
targets_j = hand_to_joints(targets, L1, L2)
n_mov = np.shape(results["joint position"])[0]
for i in range(n_mov):
    plot1trial(inputs, results, targets_j, i)

In [ ]:
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)